In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import math as mt
import datetime as dt
import random as ran
from sklearn.metrics.pairwise import euclidean_distances as euc

# 한글출력
matplotlib.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

from src.dbc import TimeDivisionKMeans
from src.dbc.utils import data_preprocessing, dimension_reduction, calc_contribution

# Before Clustering

In [3]:
# 1. load_excel
data_path = "data/apt_1.xlsx"
xlsx = pd.read_excel(data_path, header=None,
                     skiprows=2, engine="openpyxl")

# 2. data preprocessing
df = data_preprocessing(xlsx)

# 3. dimension_reduction
df = dimension_reduction(df)

# 4. day clustering
df = df[:24]
tdKMeans = TimeDivisionKMeans(df)

df.head()

,아파트1-104-1206,아파트1-104-303,아파트1-104-1307,아파트1-104-1208,아파트1-104-408,아파트1-104-203,아파트1-103-1402,아파트1-103-402,아파트1-103-1201,아파트1-103-801,...,아파트1-102-901,아파트1-103-1905,아파트1-103-503,아파트1-103-1504,아파트1-103-606,아파트1-103-903,아파트1-103-1106,아파트1-103-705,아파트1-103-1505,아파트1-103-406
2018-05-01 00:00:00,0.341,0.121,0.245,0.009,0.085,0.154,0.161,0.311,0.358,0.177,...,0.372,0.181,0.322,0.374,0.571,0.453,0.161,0.127,0.259,0.364
2018-05-01 01:00:00,0.337,0.186,0.285,0.010,0.071,0.127,0.179,0.225,0.206,0.178,...,0.245,0.238,0.292,0.382,0.327,0.352,0.172,0.133,0.234,0.286
2018-05-01 02:00:00,0.324,0.214,0.487,0.009,0.061,0.139,0.182,0.221,0.150,0.152,...,0.245,0.250,0.287,0.320,0.359,0.366,0.242,0.219,0.197,0.368
2018-05-01 03:00:00,0.319,0.250,0.300,0.009,0.067,0.132,0.172,0.274,0.142,0.205,...,0.177,0.223,0.329,0.170,0.322,0.267,0.269,0.164,0.229,0.414
2018-05-01 04:00:00,0.235,0.199,0.107,0.009,0.047,0.131,0.153,0.255,0.139,0.169,...,0.129,0.171,0.301,0.168,0.351,0.255,0.170,0.132,0.198,0.318


# Clustering Start

In [4]:
tdKMeans.init_setting()
hc, ci = tdKMeans.run()

hc

,아파트1-104-1206,아파트1-104-303,아파트1-104-1307,아파트1-104-1208,아파트1-104-408,아파트1-104-203,아파트1-103-1402,아파트1-103-402,아파트1-103-1201,아파트1-103-801,...,아파트1-102-901,아파트1-103-1905,아파트1-103-503,아파트1-103-1504,아파트1-103-606,아파트1-103-903,아파트1-103-1106,아파트1-103-705,아파트1-103-1505,아파트1-103-406
0,6.0,0.0,6.0,3.0,3.0,0.0,0.0,4.0,4.0,0.0,...,4.0,5.0,4.0,6.0,2.0,6.0,0.0,0.0,5.0,6.0
1,6.0,2.0,2.0,3.0,3.0,0.0,0.0,4.0,0.0,0.0,...,2.0,2.0,1.0,2.0,1.0,4.0,2.0,2.0,2.0,1.0
2,3.0,3.0,3.0,5.0,5.0,3.0,0.0,3.0,3.0,0.0,...,5.0,1.0,1.0,4.0,6.0,4.0,0.0,0.0,4.0,4.0
3,2.0,2.0,5.0,5.0,5.0,6.0,6.0,6.0,4.0,2.0,...,2.0,4.0,6.0,3.0,3.0,3.0,5.0,6.0,6.0,3.0
4,3.0,1.0,1.0,1.0,1.0,3.0,5.0,5.0,2.0,3.0,...,5.0,4.0,4.0,2.0,0.0,5.0,1.0,5.0,3.0,2.0
5,3.0,0.0,1.0,1.0,1.0,3.0,4.0,3.0,4.0,5.0,...,4.0,6.0,4.0,5.0,2.0,4.0,4.0,3.0,5.0,2.0
6,5.0,6.0,6.0,6.0,6.0,5.0,3.0,4.0,0.0,4.0,...,4.0,4.0,0.0,0.0,0.0,1.0,4.0,2.0,2.0,4.0
7,3.0,2.0,2.0,1.0,1.0,4.0,3.0,6.0,4.0,3.0,...,6.0,3.0,5.0,3.0,5.0,0.0,3.0,4.0,3.0,0.0


# After Clustering

# 기여도 산출

In [5]:
contribution_df = calc_contribution(hc, ci)

# 최소, 최대 사용량 가구 기여도 확인

In [6]:
# 실제(합산치 상에서의) 최소, 최대 사용량 가구
cols = df.columns

sum_df = (df.sum() * 1000).round() / 1000
min_h = cols[sum_df.argmin()]
max_h = cols[sum_df.argmax()]
print("실제 (합산치 상에서의) 최소, 최대 사용량 가구")
print("- 최소: {} ({}kwh) / 최대: {} ({}kwh)\n".format(
    min_h,
    sum_df[min_h],
    max_h,
    sum_df[max_h]
))

mean_cont_df = contribution_df.mean().round().astype("int")
_min_h = cols[mean_cont_df.argmin()]
_max_h = cols[mean_cont_df.argmax()]
print("기여도 최소, 최대 사용량 가구")
print("- 최소: {} ({}kwh) (기여도: {}%) / 최대: {} ({}kwh) (기여도: {}%)\n".format(
    _min_h,
    sum_df[min_h],
    mean_cont_df[_min_h],
    _max_h,
    sum_df[max_h],
    mean_cont_df[_max_h]
))
print("- 최소사용량 가구 기여도 , 최대사용량 가구 기여도 비교 {}배".format(round(mean_cont_df[_max_h] / mean_cont_df[_min_h])))

실제 (합산치 상에서의) 최소, 최대 사용량 가구
- 최소: 아파트1-104-1208 (0.219kwh) / 최대: 아파트1-103-703 (15.63kwh)

기여도 최소, 최대 사용량 가구
- 최소: 아파트1-104-1208 (0.219kwh) (기여도: 2%) / 최대: 아파트1-101-1403 (15.63kwh) (기여도: 25%)

- 최소사용량 가구 기여도 , 최대사용량 가구 기여도 비교 12배
